In [69]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])


if __name__ == "__main__":

    print("Load a ply point cloud, print it, and render it")
    pcd = o3d.io.read_point_cloud("MergingPointCloudData/cam_3_4.ply")
    pcd.estimate_normals()
    o3d.visualization.draw_geometries([pcd])

    print("Downsample the point cloud with a voxel of 0.02")
    voxel_down_pcd = pcd.voxel_down_sample(voxel_size=0.02)
    o3d.visualization.draw_geometries([voxel_down_pcd])

    print("Every 5th points are selected")
    uni_down_pcd = pcd.uniform_down_sample(every_k_points=5)
    o3d.visualization.draw_geometries([uni_down_pcd])

    print("Statistical oulier removal")
    cl, ind = voxel_down_pcd.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=2.0)
    display_inlier_outlier(voxel_down_pcd, ind)

    print("Radius oulier removal")
    cl, ind = voxel_down_pcd.remove_radius_outlier(nb_points=16,
                                                  radius=0.05)
    display_inlier_outlier(voxel_down_pcd, ind)

Load a ply point cloud, print it, and render it
Downsample the point cloud with a voxel of 0.02
Every 5th points are selected
Statistical oulier removal
Showing outliers (red) and inliers (gray): 
Radius oulier removal
Showing outliers (red) and inliers (gray): 


In [79]:
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        voxel_down_pcd, depth=9)
print(mesh)
o3d.visualization.draw_geometries([mesh])

run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 10541 / 10536
[Open3D DEBUG] #   Got kernel density: 0.056 (s), 330.844 (MB) / 400.621 (MB) / 535 (MB)
[Open3D DEBUG] #     Got normal field: 0.026 (s), 330.844 (MB) / 400.621 (MB) / 535 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 1.093265e-04 / 1.152410e+00
[Open3D DEBUG] #       Finalized tree: 0.0930002 (s), 335.289 (MB) / 400.621 (MB) / 535 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.138 (s), 332.098 (MB) / 400.621 (MB) / 535 (MB)
[Open3D DEBUG] #Set point constraints: 0.026 (s), 332.098 (MB) / 400.621 (MB) / 535 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 279938 / 162712 / 157217
[Open3D DEBUG] Memory Usage: 332.098 MB
[Open3D DEBUG] # Linear system solved: 0.315 (s), 332.305 (MB) / 400.621 (MB) / 535 (MB)
[Open3D DEBUG] Got average: 0.0109999 (s), 332.305 (MB) / 400.621 (MB) / 535 (MB)
[Open3D DEBUG] Iso-Value: 4.971368e-01 = 5.240319e+03 / 1.054100e+04
[Open3D DEBUG] #          To

In [80]:
print('visualize densities')
densities = np.asarray(densities)
density_colors = plt.get_cmap('plasma')(
    (densities - densities.min()) / (densities.max() - densities.min()))
density_colors = density_colors[:, :3]
density_mesh = o3d.geometry.TriangleMesh()
density_mesh.vertices = mesh.vertices
density_mesh.triangles = mesh.triangles
density_mesh.triangle_normals = mesh.triangle_normals
density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
o3d.visualization.draw_geometries([density_mesh])

visualize densities


In [81]:
print('remove low density vertices')
vertices_to_remove = densities < np.quantile(densities, 0.1)
mesh.remove_vertices_by_mask(vertices_to_remove)
print(mesh)
o3d.visualization.draw_geometries([mesh])

remove low density vertices
TriangleMesh with 10719 points and 17271 triangles.
